In [23]:
import pinecone

pinecone.init(api_key="YOUR_API_KEY", environment="us-west4-gcp-free")

In [24]:
captions = []
with open("8000.txt") as file:
    while line := file.readline():
        t = line.split("~")
        captions.append((t[0].strip(), t[1].strip()))

with open("11000.txt") as file:
    while line := file.readline():
        t = line.split("~")
        captions.append((t[0].strip(), t[1].strip()))

print(len(captions))

5000


In [25]:
captions[0]

('the road is lined with lots of trees',
 '/content/drive/MyDrive/wsl/CAM_FRONT/n015-2018-09-19-12-17-39+0800__CAM_FRONT__1537330974912460.jpg')

In [26]:
from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('all-MiniLM-L6-v2')

In [27]:
sentences = [t[0] for t in captions]

In [28]:
print(len(sentences))
sentences[0]

5000


'the road is lined with lots of trees'

In [29]:
embeddings = model.encode(sentences)

In [30]:
embeddings[0].shape

(384,)

In [31]:
# Index to perform ANN search, using euclidean distance metric.
# pinecone.create_index(name="my-index", dimension=384, metric="euclidean", index_type="approximated")   # This index_type is also the default.

In [33]:
pinecone.list_indexes()

['my-index']

In [34]:
# Connect to the index.
index = pinecone.Index(index_name="my-index")

In [11]:
upsert_data = [(str(i), [float(e) for e in embeddings[i]]) for i in range(0, len(embeddings))]

In [12]:
type(upsert_data[0][1][0])

float

In [13]:
upsert_data[1]

('1',
 [0.07265501469373703,
  0.021544551476836205,
  0.03664659708738327,
  0.05536316707730293,
  0.06005159765481949,
  -0.0292188860476017,
  0.0051211221143603325,
  -0.0735580325126648,
  0.004594874568283558,
  -0.06857388466596603,
  0.05705650895833969,
  -0.03692513704299927,
  -0.04603470489382744,
  0.06455302238464355,
  0.04950282722711563,
  0.0835762545466423,
  -0.023486655205488205,
  0.03523162007331848,
  0.07350225001573563,
  0.03251279145479202,
  -0.06647838652133942,
  0.010619957000017166,
  -0.009852875024080276,
  0.04867799952626228,
  -0.03707702085375786,
  0.02682539075613022,
  0.021085457876324654,
  0.05871135741472244,
  -0.030575696378946304,
  -0.0062711769714951515,
  0.02527913637459278,
  0.01335686445236206,
  0.10386192053556442,
  0.07754377275705338,
  -0.028456898406147957,
  0.058156609535217285,
  0.04412734881043434,
  -0.0026816795580089092,
  0.05263850465416908,
  0.028779475018382072,
  0.016317063942551613,
  -0.02794475294649601,


In [36]:
# Insert data.
# Upsert: insert a new vector, or update if same Id exists.
for i in range(0, 49):
    index.upsert(upsert_data[i*100:(i + 1)*100])

When upserting larger amounts of data, upsert data in batches of 100 vectors or fewer over multiple upsert requests.

In [35]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5900}},
 'total_vector_count': 5900}

In [17]:
# query_vector = [float(e) for e in model.encode(["a street with a tree in the corner and a street light"])[0]]

In [15]:
# query_vector

In [16]:
# # Query the DB.
# res = index.query(
#   vector=query_vector,
#   top_k=5,
#   include_values=True
# )

# res

In [17]:
# images = []

# for e in res['matches']:
#     images.append(captions[int(e['id'])][1].split("/")[-1])
#     print(sentences[int(e['id'])])
#     print(captions[int(e['id'])][1])
#     print("")

In [18]:
# base = "../data/sets/nuimages/samples/CAM_FRONT/"
# query_path = "n013-2018-09-17-13-23-26+0800__CAM_FRONT__1537162373162407.jpg"

In [19]:
# from IPython.display import display
# from PIL import Image

# display(Image.open(base + query_path))

In [20]:
# for _ in images:
#     display(Image.open(base + _))

In [21]:
# Delete an index when no longer needed.
# pinecone.delete_index("my-index")